<a href="https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/1_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines3 Giriş


Bu not defterinde, stable baselines kitaplığını kullanmanın temellerini öğreneceksiniz: bir RL modelinin nasıl oluşturulacağını, nasıl eğitileceğini ve değerlendirileceğini göreceğiz. 


```
pip install stable-baselines3[extra]
```

In [17]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

Stable-Baselines3 gym arayüzünü kullanan ortamlarda çalışır. [gym arayüzü](https://www.gymlibrary.ml/).

openai gym ile ilgili kaynak koduna [buradan](https://github.com/openai/gym) erişilebilir.

In [18]:
import gym
import numpy as np

ilk olarak stable baselines kütüphanesinden kullanacağımız RL algoritmasını içeren modeli import edeceğiz. [stable baseline algoritmaları](https://stable-baselines.readthedocs.io/en/master/)

In [19]:
from stable_baselines3 import PPO

Sonraki import edeceğimiz şey ise ilgili algortima için networkleri oluşturacak olan politika sınıfıdır. 

In [20]:
from stable_baselines3.ppo.policies import MlpPolicy

## Gym ortamının ve öznenin (agent) oluşturulması

Örnek olarak CartPole (Direk arabası) örneğini kullanacağız. 

"Sürtünmesiz bir yol boyunca hareket eden bir arabaya, tahrik edilmeyen bir mafsal tarafından bir kutup bağlanır. Sistem, arabaya +1 (sağ) veya -1 (sol) kuvveti uygulanarak kontrol edilir. Sarkaç dik başlar ve amaç düşmesini önlemektir. Direğin dik kaldığı her zaman adımı için +1 ödül verilir. "

Cartpole ortamı: [https://www.gymlibrary.ml/environments/classic_control/cart_pole/](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)


MlpPolicy'yi seçtik çünkü CartPole görevinin gözlemi görüntüler (resimler) değil bir özellik vektörüdür.

In [21]:
env = gym.make('CartPole-v1')

model = PPO(MlpPolicy, env, verbose=0)

Aşağıda özneyi (agent) değerlendirmek için bir yardımcı fonksiyon tanımlıyoruz:

In [22]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

Şimdi eğitilmemiş özneyi değerlendirelim. 

In [23]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_episodes=100)

Mean reward: 22.39 Num episodes: 100


Aslında Stable-Baselines kütüphanesinde bu değerlendirme işini yapan evaluate_policy fonksiyonu çoktan tanımlanmış. şimdi bu fonksiyonu kullanalım:

In [24]:
from stable_baselines3.common.evaluation import evaluate_policy

In [25]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:42.65 +/- 14.06


## Özneyi Eğitelim ve Değerlendirelim (Train the agent and evaluate it)

In [26]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

In [27]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:371.90 +/- 113.17


Tabiki eğitim sonrası ortalama ödül (mean reward) artacaktır ! 

### video kaydının hazırlanması

In [28]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [29]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

[VecVideoRecorder] aracını kullanarak video kaydı yapacağız. (https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder)

In [30]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

### Eğitilen öznenin görselleştirilmesi ve video gösterimi

In [31]:
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

Saving video to /content/videos/ppo2-cartpole-step-0-to-step-500.mp4


In [32]:
show_videos('videos', prefix='ppo2')